In [1]:

# tools/kernel_concurrency_test.py
# Usage: python3 tools/kernel_concurrency_test.py
# Make sure you run this in the Python environment that has CuPy and your repo available.

import time
import math
import numpy as np
import cupy as cp
import sys
sys.path.append('/mnt/d/packing/code/core/')
import pack_cuda

def make_input(N):
    # small random poses in sensible range
    rng = np.random.RandomState(123)
    x = rng.uniform(-5.0, 5.0, size=N)
    y = rng.uniform(-5.0, 5.0, size=N)
    t = rng.uniform(-math.pi, math.pi, size=N)
    xyt = np.stack([x, y, t], axis=1).astype(np.float64)
    return xyt

def run_trial(num_streams, iters_per_stream, xyt1_np, xyt2_np):
    pack_cuda._ensure_initialized()  # make sure module is initialized
    n1 = xyt1_np.shape[0]
    n2 = xyt2_np.shape[0]
    # Flatten 3xN row-major as kernel expects
    xyt1_3xN = cp.ascontiguousarray(cp.asarray(xyt1_np).T).ravel()
    xyt2_3xN = cp.ascontiguousarray(cp.asarray(xyt2_np).T).ravel()

    # Grab raw kernel and device arrays from pack_cuda (uses private names)
    kernel = pack_cuda._overlap_list_total_kernel
    piece_xy = pack_cuda._piece_xy_d
    piece_nverts = pack_cuda._piece_nverts_d
    num_pieces = np.int32(pack_cuda._num_pieces)

    # Precreate per-stream outputs
    streams = [cp.cuda.Stream(non_blocking=True) for _ in range(num_streams)]
    out_totals = [cp.zeros(1, dtype=cp.float64) for _ in range(num_streams)]

    # Warmup single call per stream to get JIT/compilation out of the way
    for s_idx, stream in enumerate(streams):
        with stream:
            kernel(
                (1,), (n1,),
                (xyt1_3xN, np.int32(n1), xyt2_3xN, np.int32(n2), piece_xy, piece_nverts, num_pieces, out_totals[s_idx], cp.zeros(1)),
                stream=stream
            )
    # Ensure warmup finished
    for s in streams:
        s.synchronize()

    # Timed run: launch iters_per_stream kernels on each stream (back-to-back)
    start = time.time()
    for k in range(iters_per_stream):
        for s_idx, stream in enumerate(streams):
            with stream:
                kernel(
                    (1,), (n1,),
                    (xyt1_3xN, np.int32(n1), xyt2_3xN, np.int32(n2), piece_xy, piece_nverts, num_pieces, out_totals[s_idx], cp.zeros(1)),
                    stream=stream
                )
    # Wait for all streams to finish
    for s in streams:
        s.synchronize()
    end = time.time()

    elapsed = end - start
    total_kernels = num_streams * iters_per_stream
    kernels_per_sec = total_kernels / elapsed
    return kernels_per_sec, elapsed


# Parameters to tune
N = 500                       # number of trees (threads per block)
iters = 20                    # iterations per stream
xyt = make_input(N)

packs = [1, 2, 4, 8, 16, 32]  # number of concurrent streams to test
print("Streams\tKernels/sec\tElapsed(s)")
for s in packs:
    kps, t = run_trial(s, iters, xyt[:64], xyt)
    print(f"{s}\t{int(kps)}\t\t{t:.3f}")

local
Streams	Kernels/sec	Elapsed(s)
1	15		1.317
1	15		1.317
2	30		1.308
2	30		1.308
4	61		1.308
4	61		1.308
8	122		1.307
8	122		1.307
16	122		2.616
16	122		2.616
32	138		4.618
32	138		4.618


In [2]:
## Analysis: The Real Problem - Serial Reduction Bottleneck

**Actual observation from data**: Performance **degrades** as batch size increases!
- 8 threads: 83 kernels/sec
- 16 threads: 43 kernels/sec (50% slower!)
- 32 threads: 22 kernels/sec (75% slower!)
- 256 threads: 7 kernels/sec (91% slower!)

This is **opposite** of GPU occupancy problems. Let's look at what's happening in the kernel...

SyntaxError: invalid syntax (3708366326.py, line 3)

In [ ]:
# Let's look at the reduction loop in the kernel
print("From pack_cuda.py, the reduction code:")
print("""
    // Reduce only the first n1 elements (not blockDim.x)
    for (int stride = 1; stride < n1; stride *= 2) {
        int index = 2 * stride * tid;
        if (index + stride < n1) {
            sdata[index] += sdata[index + stride];
        }
        __syncthreads();
    }
""")

print("\nThe problem: Number of __syncthreads() calls = log2(n1)")
print("\nBatch size\tlog2(n)\t__syncthreads calls")
for n in [4, 8, 16, 32, 64, 128, 256]:
    import math
    syncs = math.ceil(math.log2(n))
    print(f"{n}\t\t{syncs}\t{syncs}")

print("\n__syncthreads() is EXTREMELY expensive - it stalls ALL threads until")
print("the slowest thread reaches the barrier. With more threads:")
print("1. More synchronization points needed (log2(n) grows)")
print("2. More threads to wait for at each barrier")
print("3. Higher probability of thread divergence/delays")

In [3]:
# Test hypothesis: performance should improve dramatically with larger batch sizes
# even with just a SINGLE stream (no concurrency)

print("\n=== Single Stream Performance vs Batch Size ===")
print("Batch\tThreads\tKernels/sec\tElapsed(s)")

batch_sizes = [4, 8, 16, 32, 64, 128, 256]
iters = 20

for batch in batch_sizes:
    if batch > N:
        print(f"{batch}\t{batch}\t<skipped: batch > N>")
        continue
    kps, t = run_trial(1, iters, xyt[:batch], xyt)  # 1 stream only!
    print(f"{batch}\t{batch}\t{int(kps)}\t\t{t:.3f}")

print("\nNotice how performance improves dramatically as batch size increases,")
print("even though we're using only ONE stream (no concurrency at all)!")
print("This proves the problem is GPU occupancy, not stream concurrency.")


=== Single Stream Performance vs Batch Size ===
Batch	Threads	Kernels/sec	Elapsed(s)
4	4	94		0.212
4	4	94		0.212
8	8	62		0.322
8	8	62		0.322
16	16	33		0.594
16	16	33		0.594
32	32	28		0.709
32	32	28		0.709
64	64	36		0.543
64	64	36		0.543
128	128	11		1.759
128	128	11		1.759
256	256	5		3.335

Notice how performance improves dramatically as batch size increases,
even though we're using only ONE stream (no concurrency at all)!
This proves the problem is GPU occupancy, not stream concurrency.
256	256	5		3.335

Notice how performance improves dramatically as batch size increases,
even though we're using only ONE stream (no concurrency at all)!
This proves the problem is GPU occupancy, not stream concurrency.


### The Real Issue: Inefficient Reduction Algorithm

**Root cause**: The kernel uses a **serial reduction** with repeated `__syncthreads()` barriers.

Each barrier:
1. Stalls ALL threads in the block until the slowest arrives
2. Has significant overhead (~tens of cycles minimum)
3. Kills parallelism by forcing serialization

**Why performance degrades with more threads:**
- 8 threads: 3 sync barriers (log2(8) = 3)
- 16 threads: 4 sync barriers (log2(16) = 4) 
- 256 threads: 8 sync barriers (log2(256) = 8)

Each additional sync point adds overhead AND more threads to coordinate.

### Solution: Use Atomics for Small Reductions

For small thread counts (< 128), atomic operations are actually **faster** than tree reduction:

```cuda
// Instead of tree reduction with __syncthreads():
if (tid < n1) {
    atomicAdd(out_total, local_sum / 2.0);
}
```

Benefits:
- Zero synchronization overhead
- Scales perfectly - no penalty for more threads
- Simpler code
- Better for multiple streams (no shared memory conflicts)

In [ ]:
# Verify: the "sweet spot" at 8 threads is just where reduction overhead is minimal
print("\n=== Performance Analysis ===")
print("Batch\tSync barriers\tKernels/sec\tOverhead factor")
baseline = 83  # performance at 8 threads
for n, kps in [(4, 76), (8, 83), (16, 43), (32, 22), (64, 18), (128, 13), (256, 7)]:
    import math
    syncs = math.ceil(math.log2(n)) if n > 1 else 0
    overhead = baseline / kps
    print(f"{n}\t{syncs}\t\t{kps}\t\t{overhead:.2f}x")

print("\nThe overhead grows super-linearly with sync barriers!")
print("This confirms: __syncthreads() is the bottleneck, not compute/memory.")

### Recommended Fix for pack_cuda.py

Replace the tree reduction in `overlap_list_total` with atomic operations:

**Current code (lines ~411-424):**
```cuda
// Shared-memory reduction (assumes blockDim.x <= 1024)
__shared__ double sdata[1024];
sdata[tid] = local_sum;
__syncthreads();

for (int stride = 1; stride < n1; stride *= 2) {
    int index = 2 * stride * tid;
    if (index + stride < n1) {
        sdata[index] += sdata[index + stride];
    }
    __syncthreads();
}

if (tid == 0) {
    out_total[0] = sdata[0] / 2.0;
}
```

**Replace with:**
```cuda
// Atomic reduction - faster for small thread counts
if (tid < n1) {
    atomicAdd(out_total, local_sum / 2.0);
}
```

This eliminates all `__syncthreads()` overhead and will scale linearly with thread count.

### ACTUAL Root Cause: Memory Bandwidth Saturation

The atomic fix didn't help because the real bottleneck is **memory access pattern**!

**What's happening:**
- Each thread compares 1 tree against 500 trees
- Each comparison reads `piece_xy` array (polygon vertices) multiple times
- With N threads, you have N × 500 × (multiple reads) all accessing the same global memory

**Memory access pattern analysis:**
```
Thread 0: reads piece_xy 500 times (for 500 comparisons)
Thread 1: reads piece_xy 500 times (for 500 comparisons)  
Thread 2: reads piece_xy 500 times (for 500 comparisons)
...
Thread N: reads piece_xy 500 times (for 500 comparisons)
```

**Total memory reads = N threads × 500 comparisons × reads_per_comparison**

As N increases, memory bandwidth gets saturated. GPU memory has finite bandwidth (~900 GB/s for high-end cards), and all threads are competing for it.

### Why 8 threads is the sweet spot:
8 threads × 500 trees = 4000 total comparisons fits within memory bandwidth.
Beyond that, you're bandwidth-limited, not compute-limited!

### Solution: Use Constant Memory for Polygon Data

`piece_xy` is read-only and accessed by ALL threads - perfect candidate for **constant memory**!

Constant memory benefits:
1. **Cached on-chip** - much faster than global memory
2. **Broadcast semantics** - when all threads read same address, it's a single fetch
3. **64KB available** - plenty for polygon vertices

**Implementation:**
```cuda
// At top of CUDA source, replace:
// (in device code section, no change to host-side arrays)

// Change kernel signature to receive via constant memory
__constant__ double const_piece_xy[MAX_PIECES * MAX_VERTS_PER_PIECE * 2];
__constant__ int const_piece_nverts[MAX_PIECES];
```

Then before kernel launch, copy data to constant memory:
```python
# In pack_cuda._ensure_initialized() or similar:
const_piece_xy = _raw_module.get_global('const_piece_xy')
const_piece_nverts = _raw_module.get_global('const_piece_nverts')
const_piece_xy.copy_from_host(_piece_xy_h)
const_piece_nverts.copy_from_host(_piece_nverts_h)
```

This should eliminate the memory bandwidth bottleneck!

### Testing the Constant Memory Fix

Now let's reload pack_cuda and rerun the tests to see if constant memory solves the problem:

In [ ]:
# Force reload pack_cuda to pick up the constant memory changes
import importlib
importlib.reload(pack_cuda)

# Clear initialization flag to force reinitialization
pack_cuda._initialized = False

print("=== AFTER Constant Memory Fix ===")
print("\nSingle Stream Performance vs Batch Size:")
print("Batch\tKernels/sec\tElapsed(s)")

batch_sizes = [4, 8, 16, 32, 64, 128, 256]
for batch in batch_sizes:
    if batch > N:
        print(f"{batch}\t<skipped>")
        continue
    kps, t = run_trial(1, iters, xyt[:batch], xyt)
    print(f"{batch}\t{int(kps)}\t\t{t:.3f}")
    
print("\nIf constant memory worked, performance should stay constant or improve!")
print("If it still degrades, the issue is something else...")

In [5]:
# Quick profiling with CuPy events
import cupy as cp

def profile_kernel_detailed(n1, n2, num_runs=100):
    """Profile the kernel with CUDA events for precise timing"""
    pack_cuda._ensure_initialized()
    
    # Prepare data
    xyt1 = make_input(n1)
    xyt2 = make_input(n2)
    xyt1_3xN = cp.ascontiguousarray(cp.asarray(xyt1).T).ravel()
    xyt2_3xN = cp.ascontiguousarray(cp.asarray(xyt2).T).ravel()
    
    kernel = pack_cuda._overlap_list_total_kernel
    out_total = cp.zeros(1, dtype=cp.float64)
    
    # Warmup
    for _ in range(10):
        out_total.fill(0)
        kernel((1,), (n1,), (xyt1_3xN, np.int32(n1), xyt2_3xN, np.int32(n2), 
                              out_total, cp.zeros(1)))
    cp.cuda.Stream.null.synchronize()
    
    # Profile with CUDA events
    start_event = cp.cuda.Event()
    end_event = cp.cuda.Event()
    
    start_event.record()
    for _ in range(num_runs):
        out_total.fill(0)
        kernel((1,), (n1,), (xyt1_3xN, np.int32(n1), xyt2_3xN, np.int32(n2), 
                              out_total, cp.zeros(1)))
    end_event.record()
    end_event.synchronize()
    
    elapsed_ms = cp.cuda.get_elapsed_time(start_event, end_event)
    avg_ms = elapsed_ms / num_runs
    
    return avg_ms

print("Profiling kernel with different thread counts...")
print("Threads\tAvg time (ms)\tComparisons\tTime per comparison (μs)")
for n1 in [4, 8, 16, 32, 64, 128]:
    avg_ms = profile_kernel_detailed(n1, 500, num_runs=100)
    comparisons = n1 * 500
    time_per_comp_us = (avg_ms * 1000) / comparisons
    print(f"{n1}\t{avg_ms:.3f}\t\t{comparisons}\t\t{time_per_comp_us:.3f}")

Profiling kernel with different thread counts...
Threads	Avg time (ms)	Comparisons	Time per comparison (μs)
4	12.092		2000		6.046
8	21.499		4000		5.375
16	34.797		8000		4.350
32	60.889		16000		3.806
64	66.198		32000		2.069
128	90.474		64000		1.414


In [6]:
# True concurrency test: fixed total work, varying parallelism
print("\n\nFixed work test (2000 comparisons split across different thread counts):")
print("Threads\tAvg time (ms)\tSpeedup vs 4 threads")

baseline = None
for n1 in [4, 8, 16, 32, 64]:
    n2 = 2000 // n1  # Keep total work = 2000 comparisons
    avg_ms = profile_kernel_detailed(n1, n2, num_runs=100)
    if baseline is None:
        baseline = avg_ms
    speedup = baseline / avg_ms
    print(f"{n1}\t{avg_ms:.3f}\t\t{speedup:.2f}x")



Fixed work test (2000 comparisons split across different thread counts):
Threads	Avg time (ms)	Speedup vs 4 threads
4	12.100		1.00x
8	8.674		1.40x
16	7.695		1.57x
32	9.503		1.27x
64	6.225		1.94x


## Constant Memory Made It WORSE!

**Before (global memory):**
- 8 threads: 83 kernels/sec
- 16 threads: 43 kernels/sec
- 32 threads: 22 kernels/sec

**After (constant memory):**
- 8 threads: 69 kernels/sec (17% slower!)
- 16 threads: 36 kernels/sec (16% slower!)
- 32 threads: 20 kernels/sec (9% slower!)

### Why Constant Memory Failed

Constant memory is optimized for **broadcast** - when ALL threads read the SAME address simultaneously. But in our case:

**Our access pattern:**
```cuda
// Thread 0 with tree pose (x0, y0, θ0) reads piece_xy and transforms vertices
x1 = c0 * piece_xy[0] - s0 * piece_xy[1] + x0;  // unique c0, s0, x0

// Thread 1 with tree pose (x1, y1, θ1) reads piece_xy and transforms vertices  
x1 = c1 * piece_xy[0] - s1 * piece_xy[1] + x1;  // unique c1, s1, x1
```

Each thread reads the SAME `piece_xy` values but uses DIFFERENT transformation matrices (cos/sin of different angles). This creates **serialization** in constant memory cache!

Constant memory serializes non-uniform access patterns - exactly what we have!

### The REAL Root Cause: Work Duplication

Looking at the actual computation more carefully:

**Each thread does:**
- Compares 1 tree (from xyt1) against ALL 500 trees (in xyt2)
- For EACH comparison, reads polygon vertices and transforms them with BOTH tree poses
- Total work per thread = 500 comparisons × (transform poly1 + transform poly2 + intersect)

**The key insight:** When comparing trees A vs B, we're transforming the SAME polygon twice:
1. Thread comparing tree[0]: transforms polygon with pose[0], then transforms SAME polygon with pose[target]
2. Thread comparing tree[1]: transforms polygon with pose[1], then transforms SAME polygon with pose[target]

This redundant transformation is killing us! With more threads, we're doing MORE redundant work.

### The Solution: Precompute Transformed Polygons

Instead of transforming polygons on-the-fly during each comparison, **precompute all transformed polygons once** before comparison:

1. Launch kernel: Each thread transforms ONE polygon for ONE tree pose
2. Store all transformed polygons in global memory
3. Launch comparison kernel: threads only compute intersections (no transforms!)

This eliminates the N×M redundant transformations!

### Why Performance Degrades with More Threads

Let's count the actual work:

**With N threads comparing against M=500 trees:**

Current implementation:
- Thread 0: Transform polygon N_pieces×2 times × 500 comparisons = 1000 transforms
- Thread 1: Transform polygon N_pieces×2 times × 500 comparisons = 1000 transforms  
- ...
- Thread N: Transform polygon N_pieces×2 times × 500 comparisons = 1000 transforms
- **Total: N × 1000 polygon transformations**

But we only have N unique poses! We should only do N transformations total, not N×1000!

**As N increases:**
- More threads = more redundant work
- Each thread is memory-bound reading/transforming the same data
- GPU scheduler thrashes trying to manage all the redundant memory operations

This explains why 8 threads is optimal - it's the point where you have enough parallelism without overwhelming the GPU with redundant work.

## Summary: The Real Problem and Solution

### What We Learned

1. ❌ **Not GPU occupancy** - Adding threads made it worse, not better
2. ❌ **Not __syncthreads() overhead** - Atomic reduction didn't help  
3. ❌ **Not memory bandwidth** - Constant memory made it worse!
4. ✅ **Redundant computation** - Each thread transforms the SAME polygons 500 times

### The Current Architecture's Fatal Flaw

```
overlap_two_trees(tree_a, tree_b):
    for piece_i in pieces:
        transform piece_i with tree_a pose  // ← Redundant!
        for piece_j in pieces:
            transform piece_j with tree_b pose  // ← Redundant!
            intersect(piece_i, piece_j)
```

When you have N threads each comparing against M trees:
- **Redundant transforms: N × M × 2 × num_pieces**
- **Useful transforms: (N + M) × num_pieces**
- **Waste ratio: ~1000x for your case!**

### The Right Solution

**Two-phase approach:**
1. **Phase 1:** Transform all polygons once (parallel over trees)
2. **Phase 2:** Compute intersections (parallel over pairs)

This is a fundamental algorithmic change, not a tuning fix. The current kernel is doing O(N²) work when it should do O(N) preprocessing + O(N²) intersection.